Download torch and torchvision

In [ ]:
# pip install torch

In [ ]:
# pip install torchvision

Initialization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")


# Constant to control how frequently we print train loss.
print_every = 100

If the output is "using device: cpu": download CUDA toolkit (https://developer.nvidia.com/cuda-downloads)

In [ ]:
NUM_TRAIN = 1000

# data augmentation
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Split dataset -> train, val, test
button_train = dset.button('Data/images', train=True, transform=transform)
loader_train = DataLoader(button_train, batch_size= 256, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

button_val = dset.button('Data/images', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(button_val, batch_size=256,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

button_test = dset.button('Data/images', train=False, download=True,
                            transform=transform)
loader_test = DataLoader(button_test, batch_size=64)